# Módulo 5 Tarefa 1
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.). 

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo ```SINASC_RO_2019.csv``` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo ```estrutura_sinasc_para_CD.pdf``` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
``` 
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
```
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis ```['ESTCIVMAE', 'CONSULTAS']``` possuem o código ```9```, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código ```9```.<br>
6. Substitua os valores faltantes da quantitativa (```QTDFILVIVO```) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'. 
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'. 
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção ```inplace = True```.

In [1]:
import pandas as pd
import requests

# 1) LEITURA DO ARQUIVO E DISPLAY
base = pd.read_csv('SINASC_RO_2019.csv')

display(base)

,ORIGEM,CODESTAB,CODMUNNASC,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,CODOCUPMAE,QTDFILVIVO,QTDFILMORT,...,KOTELCHUCK,CONTADOR,munResStatus,munResTipo,munResNome,munResUf,munResLat,munResLon,munResAlt,munResArea
0,1,2679477.0,110001,1,19,5.0,8 a 11 anos,NaN,0.0,0.0,...,5,1,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
1,1,2679477.0,110001,1,29,2.0,8 a 11 anos,999992.0,1.0,0.0,...,5,2,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
2,1,2679477.0,110001,1,37,9.0,8 a 11 anos,513205.0,2.0,0.0,...,5,3,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
3,1,2516500.0,110001,1,30,5.0,12 anos ou mais,231205.0,0.0,0.0,...,4,4,ATIVO,MUNIC,Alto Alegre dos Parecis,Rondônia,-12.13178,-61.85308,397.0,3958.273
4,1,2516500.0,110001,1,30,2.0,8 a 11 anos,999992.0,1.0,0.0,...,5,5,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27023,1,2604388.0,510340,1,32,2.0,12 anos ou mais,333115.0,1.0,0.0,...,5,2767632,ATIVO,MUNIC,Vilhena,Rondônia,-12.74137,-60.13858,595.0,11518.915
27024,1,2752654.0,510675,1,19,1.0,8 a 11 anos,999992.0,0.0,0.0,...,5,2772678,ATIVO,MUNIC,Chupinguaia,Rondônia,-12.55684,-60.90185,340.0,5126.723
27025,1,9659366.0,510787,1,24,2.0,8 a 11 anos,421125.0,0.0,1.0,...,5,2776905,ATIVO,MUNIC,Vilhena,Rondônia,-12.74137,-60.13858,595.0,11518.915
27026,1,9659366.0,510787,1,21,2.0,8 a 11 anos,999992.0,1.0,0.0,...,5,2777105,ATIVO,MUNIC,Vilhena,Rondônia,-12.74137,-60.13858,595.0,11518.915


In [2]:
# 2) PESQUISA SE HÁ LINHAS DUPLICADAS
base.duplicated()
base.drop_duplicates().shape
# Não há duplicados

(27028, 69)

In [3]:
# 3) seu código aqui
base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27028 entries, 0 to 27027
Data columns (total 69 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ORIGEM        27028 non-null  int64  
 1   CODESTAB      26913 non-null  float64
 2   CODMUNNASC    27028 non-null  int64  
 3   LOCNASC       27028 non-null  int64  
 4   IDADEMAE      27028 non-null  int64  
 5   ESTCIVMAE     26711 non-null  float64
 6   ESCMAE        26716 non-null  object 
 7   CODOCUPMAE    24121 non-null  float64
 8   QTDFILVIVO    25455 non-null  float64
 9   QTDFILMORT    24930 non-null  float64
 10  CODMUNRES     27028 non-null  int64  
 11  GESTACAO      25796 non-null  object 
 12  GRAVIDEZ      26949 non-null  object 
 13  PARTO         26979 non-null  object 
 14  CONSULTAS     27028 non-null  int64  
 15  DTNASC        27028 non-null  object 
 16  HORANASC      27007 non-null  float64
 17  SEXO          27024 non-null  object 
 18  APGAR1        26932 non-nu

In [4]:
# 4) COLUNAS QUE NÃO POSSUEM NaN
base['GESTACAO'].isnull().sum()

1232

In [5]:
# 5) seu código aqui
base['GRAVIDEZ'].isnull().sum()

79

In [6]:
# 6) Seu código aqui
base['ESTCIVMAE'].isnull().sum() 


317

In [7]:
# 7) seu código aqui
base['ESCMAE'].isnull().sum() 


312

In [8]:
# 8) seu código aqui
base['QTDFILVIVO'].isnull().sum() 


1573

In [9]:
# 10) APGAR5
base['APGAR5'].isnull().sum()

103

In [10]:
# limpar dados NaN na coluna APGAR5
base = base.dropna(subset=['APGAR5'])
base.shape

(26925, 69)

In [11]:
#Substituição dos valores NaN pelo número 9
base['ESTCIVMAE'].fillna(value=9, inplace=True)

In [12]:
#Substituição dos valores NaN pelo número 0
base['QTDFILVIVO'].fillna(value=0, inplace=True)

In [13]:
#Substituição dos valores NaN pelo número 9
base['ESCMAE'].fillna(value=0, inplace=True)

In [14]:
#Substituição dos valores NaN pelo número 9
base['PARTO'].fillna(value=0, inplace=True)

In [15]:
base.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26925 entries, 0 to 27027
Data columns (total 69 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ORIGEM        26925 non-null  int64  
 1   CODESTAB      26859 non-null  float64
 2   CODMUNNASC    26925 non-null  int64  
 3   LOCNASC       26925 non-null  int64  
 4   IDADEMAE      26925 non-null  int64  
 5   ESTCIVMAE     26925 non-null  float64
 6   ESCMAE        26925 non-null  object 
 7   CODOCUPMAE    24031 non-null  float64
 8   QTDFILVIVO    26925 non-null  float64
 9   QTDFILMORT    24836 non-null  float64
 10  CODMUNRES     26925 non-null  int64  
 11  GESTACAO      25709 non-null  object 
 12  GRAVIDEZ      26849 non-null  object 
 13  PARTO         26925 non-null  object 
 14  CONSULTAS     26925 non-null  int64  
 15  DTNASC        26925 non-null  object 
 16  HORANASC      26904 non-null  float64
 17  SEXO          26921 non-null  object 
 18  APGAR1        26921 non-nu

In [16]:
base.loc[base['APGAR5'] <= 3, 'categoria'] = 'asfixia severa'
base.loc[(base['APGAR5'] >= 4) & (base['APGAR5'] <= 5), 'categoria'] = 'asfixia moderada'
base.loc[(base['APGAR5'] >= 6) & (base['APGAR5'] <= 7), 'categoria'] = 'asfixia leve'
base.loc[base['APGAR5'] >= 8, 'categoria'] = 'normal'

In [17]:
base['categoria'].value_counts()

normal              26463
asfixia leve          320
asfixia severa         74
asfixia moderada       68
Name: categoria, dtype: int64

In [18]:
base['categoria'].replace(['asfixia severa', 'asfixia moderada', 'asfixia leve', 'normal'],
                          ['asfixia_severa', 'asfixia_moderada', 'asfixia_leve', 'normal'], inplace=True)

In [19]:
base['categoria'].value_counts()

normal              26463
asfixia_leve          320
asfixia_severa         74
asfixia_moderada       68
Name: categoria, dtype: int64